In [39]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [46]:
spark = SparkSession.builder.appName('sparkApp').getOrCreate()

In [54]:
df = spark.read.parquet("../../data/processed/cleanedData.parquet")
df.show()

+--------------+-----------+-------------------+---------+---+------+------+----------------+----------+----+---------+--------+---------------+---------+------------+----------------+------------+---------------+--------------+------------+-------+
|Transaction_ID|Customer_ID|               City|  Country|Age|Gender|Income|Customer_Segment|      Date|Year|    Month|    Time|Total_Purchases|   Amount|Total_Amount|Product_Category|Product_Type|Shipping_Method|Payment_Method|Order_Status|Ratings|
+--------------+-----------+-------------------+---------+---+------+------+----------------+----------+----+---------+--------+---------------+---------+------------+----------------+------------+---------------+--------------+------------+-------+
|       1000043|      91680|         Fort Worth|      USA| 19|  Male|   Low|             New|2023-11-23|2023| November| 8:23:26|             10|285.67474|   2856.7476|     Electronics|  Smartphone|       Same-Day|        PayPal|   Delivered|      4|


In [55]:
df = df.groupBy("Product_Category", "Date").agg(F.sum(F.col('Total_Purchases'))).orderBy("Date", "Product_Category")
df.show(50)

+----------------+----------+--------------------+
|Product_Category|      Date|sum(Total_Purchases)|
+----------------+----------+--------------------+
|           Books|2001-01-24|                 747|
|        Clothing|2001-01-24|                 732|
|     Electronics|2001-01-24|                1062|
|         Grocery|2001-01-24|                 882|
|      Home Decor|2001-01-24|                 774|
|           Books|2001-02-24|                 954|
|        Clothing|2001-02-24|                 733|
|     Electronics|2001-02-24|                 982|
|         Grocery|2001-02-24|                 821|
|      Home Decor|2001-02-24|                 651|
|           Books|2001-03-24|                 840|
|        Clothing|2001-03-24|                 838|
|     Electronics|2001-03-24|                 868|
|         Grocery|2001-03-24|                 954|
|      Home Decor|2001-03-24|                 897|
|           Books|2001-04-24|                 775|
|        Clothing|2001-04-24|  

In [56]:
minMinDates = df.agg(
    F.min("Date").alias("min_date"),
    F.max("Date").alias("max_date")
).collect()

minDate = minMinDates[0]["min_date"]
maxDate = minMinDates[0]["max_date"]

date_series = df.select("Product_Category").distinct().crossJoin(
    spark.createDataFrame(
        [(minDate, maxDate)],
        ["min_date", "max_date"]
    )
).withColumn(
    "Date", F.explode(F.sequence(F.to_date(F.col("min_date")), F.to_date(F.col("max_date")), F.expr("INTERVAL 1 DAY")))
).select("Product_Category", "Date")

dfFilled = date_series.join(df, ["Product_Category", "Date"], "left")

window_spec = Window.partitionBy("Product_Category").orderBy("Date")

dfFilled = dfFilled.withColumn(
    "Interpolated_Purchases",
    F.avg("sum(Total_Purchases)").over(window_spec)
)

dfFilled = dfFilled.fillna({"Interpolated_Purchases": 0})

dfFilled.show()


Py4JJavaError: An error occurred while calling o745.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 112.0 failed 1 times, most recent failure: Lost task 4.0 in stage 112.0 (TID 239) (172.19.51.234 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChec